In [50]:
from equations import *
from equations_matrix import *
from solvers import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Test comparing numerical and analytical solution for X_prme

In [21]:
import numpy as np

# Set dimensions
N = 51  # number of countries
J = 3  # number of sectors

# Generate dummy arrays for the exogenous inputs
w_hat = np.ones(N)
td_prime = np.zeros(N)

# pif_hat and pim_hat are (N, N, J)
pif_hat = np.ones((N, N, J)) / N
pim_hat = np.ones((N, N, J)) / N


# Define dummy classes for ModelParams and ModelShocks
class ModelParams:
    pass

class ModelShocks:
    pass

# Create instances
mp = ModelParams()
shocks = ModelShocks()

# Fill ModelParams with attributes
mp.w0 = np.ones(N)
mp.L0 = np.ones(N)
mp.alpha = np.ones((N, J)) / J
mp.pif = np.ones((N, N, J))
mp.pim = np.ones((N, N, J)) 
mp.gamma = np.ones((N, J, J)) * 0.7 / J

# Fill ModelShocks with attributes
# Ensure tilde_tau_prime > 1 to avoid division by zero issues
shocks.tilde_tau_prime = np.random.rand(N, N, J) + 1.1

In [15]:
Xf_init = mp.alpha * (w_hat * mp.w0 * mp.L0 + td_prime)[:, None]  # shape: (N, J)
Xm_init = np.zeros_like(Xf_init)                                   # shape: (N, J)

# Call your iterative solver (solve_X_prime) using the guess:
Xf_iter, Xm_iter = solve_X_prime(w_hat, pif_hat, pim_hat, td_prime, Xf_init, Xm_init, mp, shocks, mute=False)

X_prime converged in 57 iterations


In [16]:
# Call the closed-form solver (calc_X) to get the direct solution:
Xf_closed, Xm_closed = calc_X(w_hat, pif_hat, pim_hat, td_prime, mp, shocks)

In [17]:
# Compare the results between the iterative and closed-form solutions.
print("Difference in Xf (iterative vs. closed-form):", np.linalg.norm(Xf_iter - Xf_closed))
print("Difference in Xm (iterative vs. closed-form):", np.linalg.norm(Xm_iter - Xm_closed))

Difference in Xf (iterative vs. closed-form): 4.2999443126792367e-05
Difference in Xm (iterative vs. closed-form): 5.4974185203698866e-05


# 2. HHI 

$$Q_n^s \equiv \frac{\sum_{i}(\pi_{in}^{sf}X_{in}^{sf}+\pi_{in}^{sm}X_{in}^{sm})}{1+\tau_{in}}$$
Then,HHI can be calculated as
$$HHI_n=\sum_k\left(\frac{Q_n^k}{\sum_s Q_n^s}\right)^2$$

$$\text{input component}[n, j] = \sum{k} \text{mp.gamma}[n, k, j] \times \text{log\_Pm\_hat}[n, k]$$

# 3. Test to see if this paper generalize to CP(2015)

(30, 20)